In [104]:
! pip install plotly yfinance

In [105]:
!pip install tensorflow

In [106]:
import pandas as pd
import yfinance as yf
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [107]:
import yfinance as yf
import pandas as pd
import numpy as np

# ดึงข้อมูล EUR/USD จาก Yahoo Finance
stock = yf.Ticker("Advanc.bk")
historical_data = stock.history(start="2021-01-03", end="2023-06-23")

# ลบคอลัมน์ที่ไม่จำเป็น
historical_data = historical_data.drop(columns=["Dividends", "Stock Splits"])

# แปลง index (วันที่) ให้เป็น string format MM/DD/YYYY
historical_data.index = historical_data.index.strftime('%m/%d/%Y')

# ตั้งชื่อคอลัมน์ใหม่
historical_data.columns = ["Open", "High", "Low", "Close", "Volume"]

# รีเซ็ต index เพื่อให้วันที่กลายเป็นคอลัมน์ธรรมดา
historical_data.reset_index(inplace=True)

# ตั้งชื่อคอลัมน์วันที่ใหม่ ถ้าต้องการ
historical_data.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]

# แสดงผลลัพธ์
print(historical_data)


           Date        Open        High         Low       Close    Volume
0    01/04/2021  146.760824  148.018773  146.341507  147.599457   7455500
1    01/05/2021  147.599462  150.115362  146.760828  149.696045   9815300
2    01/06/2021  150.115367  153.050583  149.276733  150.534683  14578500
3    01/07/2021  152.211940  152.211940  150.534673  151.373306   7760600
4    01/08/2021  152.211944  155.147161  151.373311  154.727844  14310800
..          ...         ...         ...         ...         ...       ...
592  06/16/2023  201.230347  202.153422  199.384197  201.230347   3487600
593  06/19/2023  200.307276  201.230351  198.461126  199.384201   1122900
594  06/20/2023  200.307272  202.153422  199.384197  201.230347   2842000
595  06/21/2023  202.153426  202.153426  199.384201  199.384201   2064600
596  06/22/2023  199.384194  199.384194  194.768820  195.691895   4350000

[597 rows x 6 columns]


In [108]:
# แปลง Date จาก string → datetime
historical_data["Date"] = pd.to_datetime(historical_data["Date"], format="%m/%d/%Y")

In [109]:
print(historical_data.dtypes)


Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object


In [110]:
from sklearn.preprocessing import MinMaxScaler

# สร้างตัวแปลง
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit และ Transform เฉพาะคอลัมน์ 'Close'
historical_data["Close_scaled"] = scaler.fit_transform(historical_data[["Close"]])


In [111]:
historical_data

,Date,Open,High,Low,Close,Volume,Close_scaled
0,2021-01-04,146.760824,148.018773,146.341507,147.599457,7455500,0.099726
1,2021-01-05,147.599462,150.115362,146.760828,149.696045,9815300,0.129244
2,2021-01-06,150.115367,153.050583,149.276733,150.534683,14578500,0.141051
3,2021-01-07,152.211940,152.211940,150.534673,151.373306,7760600,0.152858
4,2021-01-08,152.211944,155.147161,151.373311,154.727844,14310800,0.200088
...,...,...,...,...,...,...,...
592,2023-06-16,201.230347,202.153422,199.384197,201.230347,3487600,0.854806
593,2023-06-19,200.307276,201.230351,198.461126,199.384201,1122900,0.828814
594,2023-06-20,200.307272,202.153422,199.384197,201.230347,2842000,0.854806
595,2023-06-21,202.153426,202.153426,199.384201,199.384201,2064600,0.828814


In [112]:
# สร้าง DataFrame ใหม่เฉพาะวันที่และราคาปิด
closedf = historical_data[["Date", "Close"]].copy()
closedf.columns = ["date", "close"]  # เปลี่ยนชื่อคอลัมน์เพื่อความสะดวก



In [113]:
# สำรองข้อมูลก่อนเปลี่ยนแปลง
close_stock = closedf.copy()

# ลบคอลัมน์ 'date' เพราะเราจะ normalize แค่ค่าตัวเลข
del closedf['date']

# Normalize ด้วย MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler(feature_range=(0, 1))
closedf = scaler.fit_transform(np.array(closedf).reshape(-1, 1))

# ตรวจสอบรูปร่างข้อมูล
print(closedf.shape)


(597, 1)


In [114]:
import numpy as np

# สมมติ closedf เป็น numpy array shape = (n_samples, 1)
total_samples = len(closedf)
train_size = int(total_samples * 0.65)

# แบ่งข้อมูล
train_data = closedf[:train_size]
test_data = closedf[train_size:]

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


Train data shape: (388, 1)
Test data shape: (209, 1)


In [115]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)


In [116]:
time_step = 10
X, y = create_dataset(closedf, time_step)
print(X.shape)
print(y.shape)

(586, 10)
(586,)


In [117]:
# สมมติฟังก์ชัน create_dataset มีอยู่แล้ว
time_step = 15

X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

# reshape input สำหรับ LSTM/GRU
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("X_train reshaped: ", X_train.shape)
print("X_test reshaped: ", X_test.shape)


X_train:  (372, 15)
y_train:  (372,)
X_test:  (193, 15)
y_test:  (193,)
X_train reshaped:  (372, 15, 1)
X_test reshaped:  (193, 15, 1)


In [118]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [119]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 15, 32)         │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 15, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,025 (82.13 KB)

 Trainable params: 21,025 (82.13 KB)

 Non-trainable params: 0 (0.00 B)

In [120]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - loss: 0.0730 - val_loss: 0.0036
Epoch 2/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0051 - val_loss: 0.0056
Epoch 3/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0054 - val_loss: 0.0036
Epoch 4/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.0043 - val_loss: 0.0041
Epoch 5/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0052 - val_loss: 0.0071
Epoch 6/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0052 - val_loss: 0.0032
Epoch 7/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0046 - val_loss: 0.0055
Epoch 8/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0050 - val_loss: 0.0040
Epoch 9/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0047 - val_loss: 0.0088
Epoch 10/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0050 - val_loss: 0.0029
Epoch 11/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0044 - val_loss: 0.0029
Epoch 12/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/ste

In [143]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


((372, 1), (193, 1))

In [144]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [145]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.721454476668715
Train data MSE:  7.406314468580191
Test data MAE:  2.068358841762748
-------------------------------------------------------------------------------------
Test data RMSE:  2.4613953045566146
Test data MSE:  6.058466845293349
Test data MAE:  1.9793432364192025


In [146]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9893008161035416
Test data explained variance regression score: 0.9716611621454208


In [147]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9841721959381996
Test data R2 score: 0.952781855110617


In [148]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.00023617012343361368
Test data MGD:  0.00017965628827122537
----------------------------------------------------------------------
Train data MPD:  0.041526668271322946
Test data MPD:  0.03293170645881797


In [149]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (597, 1)
Test predicted data:  (597, 1)


In [150]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [151]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [152]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [153]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [154]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (372, 15, 1)
X_test:  (193, 15, 1)


In [155]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [134]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 15, 32)         │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 15, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 15, 32)         │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,377 (99.13 KB)

 Trainable params: 25,377 (99.13 KB)

 Non-trainable params: 0 (0.00 B)

In [156]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - loss: 0.0793 - val_loss: 0.0034
Epoch 2/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0057 - val_loss: 0.0047
Epoch 3/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - loss: 0.0052 - val_loss: 0.0039
Epoch 4/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.0066 - val_loss: 0.0036
Epoch 5/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0045 - val_loss: 0.0047
Epoch 6/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0050 - val_loss: 0.0029
Epoch 7/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0046 - val_loss: 0.0052
Epoch 8/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0048 - val_loss: 0.0030
Epoch 9/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0039 - val_loss: 0.0042
Epoch 10/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0034 - val_loss: 0.0030
Epoch 11/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0033 - val_loss: 0.0021
Epoch 12/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/ste

In [136]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


((372, 1), (193, 1))

In [137]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [138]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (597, 1)
Test predicted data:  (597, 1)


In [139]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [140]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [141]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})
names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [142]:
lstmgrudf=closedf.tolist()
lstmgrudf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmgrudf=scaler.inverse_transform(lstmgrudf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmgrudf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()